### ETL - DIMENSAO MATERIAL

In [3]:
import pyodbc
import pandas as pd
from dotenv import load_dotenv
import os
import runipy
import datetime
import calendar
from datetime import date
import holidays
import numpy as np

In [4]:
load_dotenv("keys.env",override=True)

True

In [5]:
driver = os.environ['DRIVER']
server = os.environ['SERVER']
database = os.environ['DATABASE']

In [6]:
### funcoes 

def conexao(driver, server, database):
    
    cnxn_str = (f"Driver={str(driver)};"
                f"Server={str(server)};"
                f"Database={str(database)};"
                "Trusted_Connection=yes;")
    cnxn = pyodbc.connect(cnxn_str)
    return cnxn


def truncate_table(driver, server, database, tabela_destino):
    
    con = conexao(driver, server, database)
    
    truncate = f"""TRUNCATE TABLE {tabela_destino} """
    
    cursor = con.cursor()
    cursor.execute(truncate)
    
    con.commit()
    cursor.close()
    
    return

In [7]:
print('Driver: ',str(driver))
print('Server: ',str(server))
print('Database: ',str(database))

Driver:  SQL Server
Server:  DESKTOP-4IH8F9C\SQLEXPRESS
Database:  pardiniteste


In [8]:
con = conexao(driver, server, database)

In [14]:
### Funcao Carregar Dados Dim MATERIAL
def carregar_dados_dim_mat(driver, server, database,dados, tabela_destino):
    
    con = conexao(driver, server, database)
  
    query = f'''INSERT INTO {tabela_destino} ''' + '''(''' + ','.join(list(dados)) + ''')''' + ''' values(''' + str('?,' * dados.shape[1])[:-1] + ''')'''
    
    cursor = con.cursor()
    print("Query Feita")
    
    try:
        for index, row in dados.iterrows():
            cursor.execute(query, row.IdMaterial, row.Material)
        con.commit()
        cursor.close()
    except:
        print('Erro na Inserção dos Dados')
        
    return

In [15]:
### DIMENSAO MATERIAL
# RUN Just one time

query = """SELECT DISTINCT Material
 FROM DATASET_1 """

try:
    new_data = pd.read_sql(query, con)
    new_data['IdMaterial'] = list(range(1,len(new_data['Material'])+1,1))
    new_data = new_data[['IdMaterial','Material']]
    df = pd.read_sql("SELECT * FROM dim_mat", con)
except:
    print("Erro na Leitura dos Dados")

if len(new_data) == 0:
    print("Tabela Vazia")
else:
    new_data = new_data[~new_data['IdMaterial'].isin(df['IdMaterial'])]
    if len(new_data) > 0:
        carregar_dados_dim_mat(driver, server, database, new_data, 'dim_mat')
        print("Insersao Concluida")
    else:
        print('Do Nothing')

Query Feita
Insersao Concluida


In [16]:
### Funcao Carregar Dados Dim SITUACAO
# RUN Just one time
def carregar_dados_dim_sit(driver, server, database,dados, tabela_destino):
    
    con = conexao(driver, server, database)
  
    query = f'''INSERT INTO {tabela_destino} ''' + '''(''' + ','.join(list(dados)) + ''')''' + ''' values(''' + str('?,' * dados.shape[1])[:-1] + ''')'''
    
    cursor = con.cursor()
    print("Query Feita")
    
    try:
        for index, row in dados.iterrows():
            cursor.execute(query, row.IdSituacao, row.SITUACAO)
        con.commit()
        cursor.close()
    except:
        print('Erro na Inserção dos Dados')
        
    return

In [17]:
### DIMENSAO SITUACAO
# RUN Just one time

query = """SELECT DISTINCT SITUACAO
 FROM DATASET_1 """

try:
    new_data = pd.read_sql(query, con)
    new_data['IdSituacao'] = list(range(1,len(new_data['SITUACAO'])+1,1))
    new_data = new_data[['IdSituacao','SITUACAO']]
    df = pd.read_sql("SELECT * FROM dim_sit", con)
except:
    print("Erro na Leitura dos Dados")

if len(new_data) == 0:
    print("Tabela Vazia")
else:
    new_data = new_data[~new_data['IdSituacao'].isin(df['IdSituacao'])]
    if len(new_data) > 0:
        carregar_dados_dim_sit(driver, server, database, new_data, 'dim_sit')
        print("Insersao Concluida")
    else:
        print('Do Nothing')

Query Feita
Insersao Concluida
